In [1]:
import pandas as pd
import numpy as np

from os.path import join as path_join

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

import warnings
warnings.filterwarnings('ignore')

In [2]:
CSV_DIR = r'../../data/Modulbank'

train = pd.read_csv(path_join(CSV_DIR, 'new_train.csv'))
test = pd.read_csv(path_join(CSV_DIR, 'new_test.csv'))

In [3]:
def write_pred(file_name, prediction):
    Y_test['0'] = prediction
    Y_test.to_csv(path_join(CSV_DIR, file_name), index=False)

In [3]:
X_train, Y_train = train.drop(columns=['0']), train['0']
X_test, Y_test = test.drop(columns=['0', 'Unnamed: 0']), test[['Unnamed: 0', '0']]

In [4]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

## xgboost

##### Roc_auc_score on
 - Validation: 0.733329
 - leaderbord: 0.74497346


In [6]:
num_rounds = 10000

In [31]:
xgb_parameters = {
    #default
#     'objective': 'reg:logistic',
    'eta': 0.1,
#     'silent': 1,
#     "nthread": 4,
    "random_seed": 42,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_leaves': 30,
    'subsample': 1,
#     'colsample_bytree': 1,
    'max_depth': 3,
    'n_estimators': 100,
    'min_child_weight': 4,
    # 'max_leaf_nodes': ,
    # 'alpha': ,
    
    #lightgbm approach
#     'tree_method': 'hist',
    'grow_policy': 'lossguide'
}

xgb_train = xgb.DMatrix(X_train, Y_train, feature_names=X_train.columns)
xgb_test = xgb.DMatrix(X_test, feature_names=X_train.columns)

In [32]:
%%time
results = xgb.cv(xgb_parameters, 
                 xgb_train, 
                 num_rounds, 
                 early_stopping_rounds=10,
                 folds=skf, 
                 verbose_eval=1
                )

[0]	train-auc:0.711774+0.004383	test-auc:0.676734+0.00514366
[1]	train-auc:0.732509+0.00416342	test-auc:0.694665+0.00565428
[2]	train-auc:0.742089+0.00183125	test-auc:0.701061+0.0047952
[3]	train-auc:0.748017+0.00264989	test-auc:0.704931+0.00488604
[4]	train-auc:0.75254+0.00146404	test-auc:0.708519+0.00565829
[5]	train-auc:0.755721+0.00103385	test-auc:0.709367+0.00499013
[6]	train-auc:0.758258+0.00106744	test-auc:0.711143+0.00568527
[7]	train-auc:0.764989+0.000451892	test-auc:0.713768+0.00441113
[8]	train-auc:0.76869+0.000437921	test-auc:0.715569+0.00440005
[9]	train-auc:0.772153+0.00119416	test-auc:0.717295+0.00470011
[10]	train-auc:0.776276+0.000814117	test-auc:0.719176+0.00403793
[11]	train-auc:0.779494+0.00134516	test-auc:0.719634+0.00355202
[12]	train-auc:0.782614+0.0017668	test-auc:0.720512+0.00386975
[13]	train-auc:0.785686+0.000874823	test-auc:0.721214+0.00355295
[14]	train-auc:0.788338+0.000827988	test-auc:0.721536+0.00304806
[15]	train-auc:0.792194+0.000254327	test-auc:0.7223

In [8]:
xgb_parameters = {
    #default
#     'objective': 'reg:logistic',
    'learning_rate': 0.1,
#     'silent': 1,
#     "nthread": 4,
    "random_seed": 42,
#     "eval_metric": 'auc',
    
    # regularization parameters
    'max_leaves': 30,
    'subsample': 1,
#     'colsample_bytree': 1,
    'max_depth': 3,
    'n_estimators': 100,
    'min_child_weight': 4,
    # 'max_leaf_nodes': ,
    # 'alpha': ,
    
    #lightgbm approach
#     'tree_method': 'hist',
    'grow_policy': 'lossguide'
}


xgb_model = xgb.XGBClassifier(**xgb_parameters)
xgb_model.fit(X_train, Y_train)

In [20]:
xgb_prediction = xgb_model.predict_proba(X_test)[:, 1]

In [21]:
min(xgb_prediction)

0.01603448

In [167]:
write_pred('submission_XGBoost.csv', xgb_prediction)

## LightGBM

##### Roc_auc_score on
 - Validation: 0.736433
 - leaderbord: 0.75919661

In [5]:
lgb_parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
#     'colsample_bytree': 0.1,
#     'subsample': 0.5,
    'subsample_freq': 50,
#     'max_bin': 20,
#     'num_leaves': 100,
#     'min_data_in_leaf': 20,
    'max_depth': 8,
}

lgb_train = lgb.Dataset(X_train, label=Y_train, free_raw_data=False)
lgb_test = lgb.Dataset(X_test, label=Y_test, free_raw_data=False)

In [7]:
%%time
result = lgb.cv(lgb_parameters, 
                lgb_train, 
                num_rounds,  
                folds=skf.split(X_train, Y_train), 
                early_stopping_rounds=10, 
                verbose_eval=1, 
               )

[1]	cv_agg's auc: 0.682753 + 0.00768274
[2]	cv_agg's auc: 0.686505 + 0.00789658
[3]	cv_agg's auc: 0.693925 + 0.00868877
[4]	cv_agg's auc: 0.697117 + 0.00740897
[5]	cv_agg's auc: 0.7019 + 0.00686951
[6]	cv_agg's auc: 0.703444 + 0.00638934
[7]	cv_agg's auc: 0.708855 + 0.00405052
[8]	cv_agg's auc: 0.70955 + 0.00313817
[9]	cv_agg's auc: 0.711242 + 0.00297718
[10]	cv_agg's auc: 0.713134 + 0.0024687
[11]	cv_agg's auc: 0.714191 + 0.00222451
[12]	cv_agg's auc: 0.715755 + 0.0027071
[13]	cv_agg's auc: 0.716514 + 0.00223161
[14]	cv_agg's auc: 0.717923 + 0.00282967
[15]	cv_agg's auc: 0.71921 + 0.00234396
[16]	cv_agg's auc: 0.720966 + 0.00208003
[17]	cv_agg's auc: 0.721835 + 0.00237451
[18]	cv_agg's auc: 0.722469 + 0.00219401
[19]	cv_agg's auc: 0.723227 + 0.00188095
[20]	cv_agg's auc: 0.723923 + 0.00189091
[21]	cv_agg's auc: 0.724973 + 0.00185118
[22]	cv_agg's auc: 0.725897 + 0.00134665
[23]	cv_agg's auc: 0.726584 + 0.00149189
[24]	cv_agg's auc: 0.727574 + 0.000592281
[25]	cv_agg's auc: 0.728247 + 

In [328]:
%%time
def check_train_score(params, lgb_data, data, target, skf, num_rounds):
    roc_auc = []
    
    for train, val in skf.split(data, target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        
        temp_model = lgb.train(params, 
                               temp_lgb_train, 
                               num_rounds, 
                               verbose_eval=num_rounds
                              )
        roc_auc.append([roc_auc_score(target[train], temp_model.predict(data.loc[train])), 
                        roc_auc_score(target[val], temp_model.predict(data.loc[val]))]
                      )
        
    return np.mean(roc_auc, axis=0)

res = check_train_score(lgb_parameters, lgb_train, X_train, Y_train, skf, 40)
print(res)

[0.82220986 0.73491834]
CPU times: user 10.9 s, sys: 391 ms, total: 11.3 s
Wall time: 3.64 s


In [10]:
lgb_model = lgb.LGBMClassifier(**lgb_parameters)
lgb_model.fit(X_train, Y_train)

In [24]:
lgb_prediction = lgb_model.predict_proba(X_test)[:, 1]

In [25]:
min(lgb_prediction)

0.004435514166250044

In [332]:
write_pred('submission_Lightgbm.csv', lgb_prediction)

## CatBoost

##### Roc_auc_score on
 - Validation: 0.73722340193
 - leaderbord: 0.75641656

In [11]:
ctb_parameters = {
    #default
#     'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_type': 'Iter',
    'od_wait': 30,
    
    #regularization
    'rsm': 0.8,
    'l2_leaf_reg': 2
    
}

ctb_train = ctb.Pool(X_train, Y_train)
ctb_test = ctb.Pool(X_test, Y_test)

In [156]:
result = ctb.cv(ctb_train, 
                ctb_parameters, 
                stratified=True, 
                seed=42, 
                logging_level='Verbose'
               )

0:	learn: 0.6868207	test: 0.6829430	best: 0.6829430 (0)	total: 325ms	remaining: 5m 24s
1:	learn: 0.7079173	test: 0.7043168	best: 0.7043168 (1)	total: 765ms	remaining: 6m 21s
2:	learn: 0.7142791	test: 0.7083530	best: 0.7083530 (2)	total: 1.1s	remaining: 6m 6s
3:	learn: 0.7148700	test: 0.7069028	best: 0.7083530 (2)	total: 1.42s	remaining: 5m 54s
4:	learn: 0.7184104	test: 0.7081943	best: 0.7083530 (2)	total: 1.79s	remaining: 5m 55s
5:	learn: 0.7218008	test: 0.7117686	best: 0.7117686 (5)	total: 2.1s	remaining: 5m 47s
6:	learn: 0.7289545	test: 0.7189282	best: 0.7189282 (6)	total: 2.41s	remaining: 5m 42s
7:	learn: 0.7343653	test: 0.7269694	best: 0.7269694 (7)	total: 2.78s	remaining: 5m 44s
8:	learn: 0.7350927	test: 0.7269930	best: 0.7269930 (8)	total: 3.12s	remaining: 5m 43s
9:	learn: 0.7372088	test: 0.7303446	best: 0.7303446 (9)	total: 3.64s	remaining: 6m
10:	learn: 0.7374235	test: 0.7299437	best: 0.7303446 (9)	total: 3.96s	remaining: 5m 56s
11:	learn: 0.7383674	test: 0.7317697	best: 0.7317

93:	learn: 0.7457252	test: 0.7376712	best: 0.7377982 (74)	total: 32.5s	remaining: 5m 13s
94:	learn: 0.7457524	test: 0.7376393	best: 0.7377982 (74)	total: 32.9s	remaining: 5m 13s
95:	learn: 0.7458534	test: 0.7376197	best: 0.7377982 (74)	total: 33.2s	remaining: 5m 12s
96:	learn: 0.7460196	test: 0.7376225	best: 0.7377982 (74)	total: 33.5s	remaining: 5m 11s
97:	learn: 0.7460469	test: 0.7376047	best: 0.7377982 (74)	total: 33.8s	remaining: 5m 11s
98:	learn: 0.7461116	test: 0.7376423	best: 0.7377982 (74)	total: 34.2s	remaining: 5m 10s
99:	learn: 0.7462719	test: 0.7376293	best: 0.7377982 (74)	total: 34.5s	remaining: 5m 10s
100:	learn: 0.7463898	test: 0.7375878	best: 0.7377982 (74)	total: 34.8s	remaining: 5m 9s
101:	learn: 0.7464866	test: 0.7375985	best: 0.7377982 (74)	total: 35.1s	remaining: 5m 9s
102:	learn: 0.7466040	test: 0.7376532	best: 0.7377982 (74)	total: 35.5s	remaining: 5m 8s
103:	learn: 0.7466871	test: 0.7376611	best: 0.7377982 (74)	total: 35.8s	remaining: 5m 8s
104:	learn: 0.7467784

In [157]:
for train, val in skf.split(X_train, Y_train):
    
    temp_ctb_train = ctb.Pool(X_train.iloc[train], Y_train.iloc[train])
    temp_ctb_val = ctb.Pool(X_train.iloc[val], Y_train.iloc[val])
    
    temp_model = ctb.train(temp_ctb_train, 
                           ctb_parameters, 
                           logging_level='Verbose', 
                           eval_set=temp_ctb_val
                          )

0:	learn: 0.7172739	test: 0.7132078	best: 0.7132078 (0)	total: 87.7ms	remaining: 1m 27s
1:	learn: 0.7258904	test: 0.7217885	best: 0.7217885 (1)	total: 187ms	remaining: 1m 33s
2:	learn: 0.7271076	test: 0.7234273	best: 0.7234273 (2)	total: 291ms	remaining: 1m 36s
3:	learn: 0.7310713	test: 0.7271116	best: 0.7271116 (3)	total: 385ms	remaining: 1m 35s
4:	learn: 0.7336289	test: 0.7291816	best: 0.7291816 (4)	total: 492ms	remaining: 1m 37s
5:	learn: 0.7347214	test: 0.7296359	best: 0.7296359 (5)	total: 586ms	remaining: 1m 37s
6:	learn: 0.7358639	test: 0.7321547	best: 0.7321547 (6)	total: 701ms	remaining: 1m 39s
7:	learn: 0.7368503	test: 0.7346118	best: 0.7346118 (7)	total: 804ms	remaining: 1m 39s
8:	learn: 0.7376768	test: 0.7344081	best: 0.7346118 (7)	total: 907ms	remaining: 1m 39s
9:	learn: 0.7378872	test: 0.7337678	best: 0.7346118 (7)	total: 1s	remaining: 1m 39s
10:	learn: 0.7377547	test: 0.7346703	best: 0.7346703 (10)	total: 1.13s	remaining: 1m 41s
11:	learn: 0.7381902	test: 0.7355577	best: 

93:	learn: 0.7460038	test: 0.7373544	best: 0.7375424 (84)	total: 11s	remaining: 1m 46s
94:	learn: 0.7461245	test: 0.7374072	best: 0.7375424 (84)	total: 11.4s	remaining: 1m 48s
95:	learn: 0.7462063	test: 0.7373940	best: 0.7375424 (84)	total: 11.5s	remaining: 1m 48s
96:	learn: 0.7463267	test: 0.7374141	best: 0.7375424 (84)	total: 11.7s	remaining: 1m 49s
97:	learn: 0.7464441	test: 0.7373662	best: 0.7375424 (84)	total: 11.9s	remaining: 1m 49s
98:	learn: 0.7465660	test: 0.7373022	best: 0.7375424 (84)	total: 12s	remaining: 1m 49s
99:	learn: 0.7466039	test: 0.7373106	best: 0.7375424 (84)	total: 12.2s	remaining: 1m 49s
100:	learn: 0.7466667	test: 0.7373100	best: 0.7375424 (84)	total: 12.4s	remaining: 1m 50s
101:	learn: 0.7468854	test: 0.7373122	best: 0.7375424 (84)	total: 12.7s	remaining: 1m 52s
102:	learn: 0.7469368	test: 0.7373349	best: 0.7375424 (84)	total: 12.9s	remaining: 1m 52s
103:	learn: 0.7470229	test: 0.7373056	best: 0.7375424 (84)	total: 13.1s	remaining: 1m 52s
104:	learn: 0.7471216

69:	learn: 0.7441919	test: 0.7346636	best: 0.7346636 (69)	total: 11.4s	remaining: 2m 31s
70:	learn: 0.7442743	test: 0.7347523	best: 0.7347523 (70)	total: 11.5s	remaining: 2m 31s
71:	learn: 0.7442976	test: 0.7347813	best: 0.7347813 (71)	total: 11.7s	remaining: 2m 31s
72:	learn: 0.7441616	test: 0.7348342	best: 0.7348342 (72)	total: 11.9s	remaining: 2m 31s
73:	learn: 0.7441817	test: 0.7347986	best: 0.7348342 (72)	total: 12.1s	remaining: 2m 31s
74:	learn: 0.7441697	test: 0.7348687	best: 0.7348687 (74)	total: 12.2s	remaining: 2m 30s
75:	learn: 0.7441680	test: 0.7348664	best: 0.7348687 (74)	total: 12.4s	remaining: 2m 30s
76:	learn: 0.7442964	test: 0.7348202	best: 0.7348687 (74)	total: 12.5s	remaining: 2m 30s
77:	learn: 0.7442252	test: 0.7348683	best: 0.7348687 (74)	total: 12.7s	remaining: 2m 29s
78:	learn: 0.7442900	test: 0.7348731	best: 0.7348731 (78)	total: 12.8s	remaining: 2m 29s
79:	learn: 0.7443743	test: 0.7348814	best: 0.7348814 (79)	total: 13s	remaining: 2m 29s
80:	learn: 0.7444187	te

28:	learn: 0.7418299	test: 0.7365023	best: 0.7365023 (28)	total: 4.51s	remaining: 2m 31s
29:	learn: 0.7417881	test: 0.7362159	best: 0.7365023 (28)	total: 4.64s	remaining: 2m 30s
30:	learn: 0.7419326	test: 0.7363591	best: 0.7365023 (28)	total: 4.77s	remaining: 2m 29s
31:	learn: 0.7419910	test: 0.7369964	best: 0.7369964 (31)	total: 4.9s	remaining: 2m 28s
32:	learn: 0.7420863	test: 0.7373007	best: 0.7373007 (32)	total: 5.06s	remaining: 2m 28s
33:	learn: 0.7421148	test: 0.7372680	best: 0.7373007 (32)	total: 5.25s	remaining: 2m 29s
34:	learn: 0.7421489	test: 0.7373331	best: 0.7373331 (34)	total: 5.45s	remaining: 2m 30s
35:	learn: 0.7422290	test: 0.7373975	best: 0.7373975 (35)	total: 5.63s	remaining: 2m 30s
36:	learn: 0.7422294	test: 0.7373052	best: 0.7373975 (35)	total: 5.78s	remaining: 2m 30s
37:	learn: 0.7422615	test: 0.7374231	best: 0.7374231 (37)	total: 5.91s	remaining: 2m 29s
38:	learn: 0.7426333	test: 0.7374171	best: 0.7374231 (37)	total: 6.07s	remaining: 2m 29s
39:	learn: 0.7428321	t

In [159]:
np.mean([0.737542407, 0.7352762562, 0.7388515426])

0.7372234019333334

In [12]:
ctb_model = ctb.CatBoostClassifier(**ctb_parameters)
ctb_model.fit(X_train, Y_train)

In [161]:
ctb_prediction = ctb_model.predict_proba(ctb_test)[:, 1]

In [162]:
min(ctb_prediction)

0.01751602339353997

In [163]:
write_pred('submission_Catbooster.csv', ctb_prediction)

## Ensemble

##### Roc_auc_score on
 - Validation: 0.73638163
 - leaderbord: 0.76001779

In [40]:
%%time

num_rounds = 50
roc_auc = []

for train, val in skf.split(X_train, Y_train):

    temp_xgb_train = xgb_train.slice(train)
    temp_xgb_val = xgb_train.slice(val)

    temp_lgb_train = lgb_train.subset(train)
    temp_lgb_val = lgb_train.subset(val)


    temp_xgb_model = xgb.train(xgb_parameters, 
                                temp_xgb_train, 
                                num_rounds, 
                                verbose_eval=num_rounds
                              )
    
    temp_lgb_model = lgb.train(lgb_parameters, 
                                temp_lgb_train, 
                                num_rounds, 
                                verbose_eval=num_rounds
                          )
    
    roc_auc.append([roc_auc_score(Y_train.iloc[train], 
                                  temp_xgb_model.predict(temp_xgb_train) * 0.15 + 
                                  temp_lgb_model.predict(X_train.iloc[train]) * 0.85
                                 ),
                    roc_auc_score(Y_train.iloc[val],
                                  temp_xgb_model.predict(temp_xgb_val) * 0.15 + 
                                  temp_lgb_model.predict(X_train.iloc[val]) * 0.85
                                 )
                  ])

result = np.mean(roc_auc, axis=0)

print(result)

[0.8327191  0.73638163]
CPU times: user 46.8 s, sys: 1.02 s, total: 47.8 s
Wall time: 13.7 s


In [41]:
xgb_prediction = xgb_model.predict_proba(X_test)[:, 1]
lgb_prediction = lgb_model.predict_proba(X_test)[:, 1]
ensemble_prediction = xgb_prediction * 0.15 + lgb_prediction * 0.85

In [46]:
min(ensemble_prediction)

0.0062459306889773225

In [45]:
write_pred('submission_Ensemble.csv', ensemble_prediction)

## Stacking

##### Roc_auc_score on
 - Validation: 0.7284623610222036
 - leaderbord: 0.75707522

#### validation

In [164]:
%%time

num_rounds = 50
roc_auc = []

for train, val in skf.split(X_train, Y_train):
    
    # new simple dataFrames
    
    temp_x_train = X_train.iloc[train]
    temp_y_train = Y_train.iloc[train]
    
    temp_x_val = X_train.iloc[val]
    temp_y_val =  Y_train.iloc[val]
    
    xgb_predictions = []
    lgb_predictions = []

    for train_, val_ in skf.split(temp_x_train, temp_y_train):    

        # xgb and lgb boosters
        
        xgb_model.fit(temp_x_train.iloc[train_], temp_y_train.iloc[train_])
        lgb_model.fit(temp_x_train.iloc[train_], temp_y_train.iloc[train_])

        xgb_prediction = xgb_model.predict_proba(temp_x_train.iloc[val_])[:, 1]
        lgb_prediction = lgb_model.predict_proba(temp_x_train.iloc[val_])[:, 1]

        xgb_predictions.append(xgb_prediction)
        lgb_predictions.append(lgb_prediction)
        
    # catbooster

    temp_x_train['xgb_preds'] = 0
    temp_x_train['lgb_preds'] = 0

    for i, fold in enumerate(skf.split(temp_x_train, temp_y_train)):
        train_, val_ = fold[0], fold[1]

        temp_x_train.iloc[val_, -2] = xgb_predictions[i]

        temp_x_train.iloc[val_, -1] = lgb_predictions[i]
    
    temp_xgb_prediction = xgb_model.predict_proba(temp_x_val)[:, 1]
    temp_lgb_prediction = lgb_model.predict_proba(temp_x_val)[:, 1]
    
    temp_x_val['xgb_preds'] = temp_xgb_prediction
    temp_x_val['lgb_preds'] = temp_xgb_prediction

    ctb_model.fit(temp_x_train, temp_y_train)

    ctb_prediction = ctb_model.predict_proba(temp_x_val)[:, 1]

    roc_auc.append(roc_auc_score(temp_y_val, ctb_prediction))
    
result = np.mean(roc_auc, axis=0)

print(result)

0:	learn: 0.6462176	total: 106ms	remaining: 1m 45s
1:	learn: 0.6750464	total: 225ms	remaining: 1m 52s
2:	learn: 0.6760722	total: 367ms	remaining: 2m 1s
3:	learn: 0.6892980	total: 504ms	remaining: 2m 5s
4:	learn: 0.7179133	total: 637ms	remaining: 2m 6s
5:	learn: 0.7213574	total: 783ms	remaining: 2m 9s
6:	learn: 0.7210448	total: 927ms	remaining: 2m 11s
7:	learn: 0.7224322	total: 1.05s	remaining: 2m 10s
8:	learn: 0.7231941	total: 1.19s	remaining: 2m 10s
9:	learn: 0.7260598	total: 1.31s	remaining: 2m 9s
10:	learn: 0.7271082	total: 1.44s	remaining: 2m 9s
11:	learn: 0.7278344	total: 1.57s	remaining: 2m 9s
12:	learn: 0.7304766	total: 1.73s	remaining: 2m 11s
13:	learn: 0.7322159	total: 1.86s	remaining: 2m 11s
14:	learn: 0.7325230	total: 2.01s	remaining: 2m 11s
15:	learn: 0.7344504	total: 2.12s	remaining: 2m 10s
16:	learn: 0.7348787	total: 2.23s	remaining: 2m 9s
17:	learn: 0.7350959	total: 2.38s	remaining: 2m 9s
18:	learn: 0.7353822	total: 2.52s	remaining: 2m 10s
19:	learn: 0.7357411	total: 2.6

160:	learn: 0.7517704	total: 22.3s	remaining: 1m 56s
161:	learn: 0.7520321	total: 22.5s	remaining: 1m 56s
162:	learn: 0.7520992	total: 22.7s	remaining: 1m 56s
163:	learn: 0.7522199	total: 22.8s	remaining: 1m 56s
164:	learn: 0.7523507	total: 22.9s	remaining: 1m 56s
165:	learn: 0.7525151	total: 23.1s	remaining: 1m 55s
166:	learn: 0.7526160	total: 23.2s	remaining: 1m 55s
167:	learn: 0.7527496	total: 23.3s	remaining: 1m 55s
168:	learn: 0.7528372	total: 23.4s	remaining: 1m 55s
169:	learn: 0.7529021	total: 23.6s	remaining: 1m 55s
170:	learn: 0.7530648	total: 23.7s	remaining: 1m 55s
171:	learn: 0.7531242	total: 23.9s	remaining: 1m 54s
172:	learn: 0.7533668	total: 24s	remaining: 1m 54s
173:	learn: 0.7534862	total: 24.1s	remaining: 1m 54s
174:	learn: 0.7537008	total: 24.3s	remaining: 1m 54s
175:	learn: 0.7537130	total: 24.4s	remaining: 1m 54s
176:	learn: 0.7538559	total: 24.6s	remaining: 1m 54s
177:	learn: 0.7540579	total: 24.8s	remaining: 1m 54s
178:	learn: 0.7541238	total: 25s	remaining: 1m 5

316:	learn: 0.7736188	total: 43.9s	remaining: 1m 34s
317:	learn: 0.7737200	total: 44.1s	remaining: 1m 34s
318:	learn: 0.7738975	total: 44.2s	remaining: 1m 34s
319:	learn: 0.7739876	total: 44.4s	remaining: 1m 34s
320:	learn: 0.7741783	total: 44.5s	remaining: 1m 34s
321:	learn: 0.7742902	total: 44.7s	remaining: 1m 34s
322:	learn: 0.7744231	total: 44.8s	remaining: 1m 34s
323:	learn: 0.7746316	total: 45s	remaining: 1m 33s
324:	learn: 0.7747524	total: 45.1s	remaining: 1m 33s
325:	learn: 0.7749624	total: 45.3s	remaining: 1m 33s
326:	learn: 0.7750664	total: 45.4s	remaining: 1m 33s
327:	learn: 0.7752842	total: 45.5s	remaining: 1m 33s
328:	learn: 0.7755263	total: 45.7s	remaining: 1m 33s
329:	learn: 0.7756637	total: 45.8s	remaining: 1m 32s
330:	learn: 0.7758023	total: 46s	remaining: 1m 32s
331:	learn: 0.7760273	total: 46.1s	remaining: 1m 32s
332:	learn: 0.7761589	total: 46.3s	remaining: 1m 32s
333:	learn: 0.7762673	total: 46.4s	remaining: 1m 32s
334:	learn: 0.7764908	total: 46.5s	remaining: 1m 3

473:	learn: 0.7998732	total: 1m 5s	remaining: 1m 12s
474:	learn: 0.8000196	total: 1m 5s	remaining: 1m 12s
475:	learn: 0.8001125	total: 1m 5s	remaining: 1m 11s
476:	learn: 0.8001895	total: 1m 5s	remaining: 1m 11s
477:	learn: 0.8002831	total: 1m 5s	remaining: 1m 11s
478:	learn: 0.8005189	total: 1m 5s	remaining: 1m 11s
479:	learn: 0.8006551	total: 1m 5s	remaining: 1m 11s
480:	learn: 0.8008256	total: 1m 5s	remaining: 1m 11s
481:	learn: 0.8009938	total: 1m 5s	remaining: 1m 10s
482:	learn: 0.8011647	total: 1m 6s	remaining: 1m 10s
483:	learn: 0.8012717	total: 1m 6s	remaining: 1m 10s
484:	learn: 0.8013828	total: 1m 6s	remaining: 1m 10s
485:	learn: 0.8015736	total: 1m 6s	remaining: 1m 10s
486:	learn: 0.8016930	total: 1m 6s	remaining: 1m 10s
487:	learn: 0.8019512	total: 1m 6s	remaining: 1m 9s
488:	learn: 0.8020977	total: 1m 6s	remaining: 1m 9s
489:	learn: 0.8022109	total: 1m 6s	remaining: 1m 9s
490:	learn: 0.8023592	total: 1m 7s	remaining: 1m 9s
491:	learn: 0.8025216	total: 1m 7s	remaining: 1m 9

629:	learn: 0.8216825	total: 1m 27s	remaining: 51.5s
630:	learn: 0.8217562	total: 1m 27s	remaining: 51.4s
631:	learn: 0.8219777	total: 1m 27s	remaining: 51.2s
632:	learn: 0.8221497	total: 1m 28s	remaining: 51.1s
633:	learn: 0.8224120	total: 1m 28s	remaining: 50.9s
634:	learn: 0.8225277	total: 1m 28s	remaining: 50.8s
635:	learn: 0.8226509	total: 1m 28s	remaining: 50.6s
636:	learn: 0.8227709	total: 1m 28s	remaining: 50.5s
637:	learn: 0.8230150	total: 1m 28s	remaining: 50.3s
638:	learn: 0.8231109	total: 1m 28s	remaining: 50.2s
639:	learn: 0.8233428	total: 1m 28s	remaining: 50s
640:	learn: 0.8234960	total: 1m 29s	remaining: 49.8s
641:	learn: 0.8237550	total: 1m 29s	remaining: 49.7s
642:	learn: 0.8238390	total: 1m 29s	remaining: 49.5s
643:	learn: 0.8239019	total: 1m 29s	remaining: 49.4s
644:	learn: 0.8239679	total: 1m 29s	remaining: 49.2s
645:	learn: 0.8240611	total: 1m 29s	remaining: 49.1s
646:	learn: 0.8242656	total: 1m 29s	remaining: 49s
647:	learn: 0.8244087	total: 1m 29s	remaining: 48.

...

355:	learn: 0.7783988	total: 56.7s	remaining: 1m 42s
356:	learn: 0.7786219	total: 56.8s	remaining: 1m 42s
357:	learn: 0.7788031	total: 57s	remaining: 1m 42s
358:	learn: 0.7789976	total: 57.1s	remaining: 1m 41s
359:	learn: 0.7792058	total: 57.3s	remaining: 1m 41s
360:	learn: 0.7793346	total: 57.4s	remaining: 1m 41s
361:	learn: 0.7795147	total: 57.6s	remaining: 1m 41s
362:	learn: 0.7797381	total: 57.7s	remaining: 1m 41s
363:	learn: 0.7799814	total: 57.9s	remaining: 1m 41s
364:	learn: 0.7801224	total: 58s	remaining: 1m 40s
365:	learn: 0.7803637	total: 58.2s	remaining: 1m 40s
366:	learn: 0.7804407	total: 58.3s	remaining: 1m 40s
367:	learn: 0.7805951	total: 58.5s	remaining: 1m 40s
368:	learn: 0.7807397	total: 58.6s	remaining: 1m 40s
369:	learn: 0.7809504	total: 58.8s	remaining: 1m 40s
370:	learn: 0.7811374	total: 58.9s	remaining: 1m 39s
371:	learn: 0.7813636	total: 59.1s	remaining: 1m 39s
372:	learn: 0.7816153	total: 59.2s	remaining: 1m 39s
373:	learn: 0.7817574	total: 59.4s	remaining: 1m 3

511:	learn: 0.8045676	total: 1m 15s	remaining: 1m 12s
512:	learn: 0.8047130	total: 1m 16s	remaining: 1m 12s
513:	learn: 0.8048275	total: 1m 16s	remaining: 1m 12s
514:	learn: 0.8049916	total: 1m 16s	remaining: 1m 11s
515:	learn: 0.8050637	total: 1m 16s	remaining: 1m 11s
516:	learn: 0.8052419	total: 1m 16s	remaining: 1m 11s
517:	learn: 0.8053979	total: 1m 16s	remaining: 1m 11s
518:	learn: 0.8055469	total: 1m 16s	remaining: 1m 11s
519:	learn: 0.8058238	total: 1m 16s	remaining: 1m 10s
520:	learn: 0.8059533	total: 1m 16s	remaining: 1m 10s
521:	learn: 0.8060959	total: 1m 17s	remaining: 1m 10s
522:	learn: 0.8062244	total: 1m 17s	remaining: 1m 10s
523:	learn: 0.8064078	total: 1m 17s	remaining: 1m 10s
524:	learn: 0.8065758	total: 1m 17s	remaining: 1m 10s
525:	learn: 0.8067288	total: 1m 17s	remaining: 1m 9s
526:	learn: 0.8068451	total: 1m 17s	remaining: 1m 9s
527:	learn: 0.8069714	total: 1m 17s	remaining: 1m 9s
528:	learn: 0.8070126	total: 1m 17s	remaining: 1m 9s
529:	learn: 0.8071423	total: 1m 

667:	learn: 0.8246781	total: 1m 37s	remaining: 48.3s
668:	learn: 0.8247988	total: 1m 37s	remaining: 48.2s
669:	learn: 0.8248902	total: 1m 37s	remaining: 48s
670:	learn: 0.8249597	total: 1m 37s	remaining: 47.8s
671:	learn: 0.8250751	total: 1m 37s	remaining: 47.7s
672:	learn: 0.8251966	total: 1m 37s	remaining: 47.6s
673:	learn: 0.8252963	total: 1m 38s	remaining: 47.4s
674:	learn: 0.8254303	total: 1m 38s	remaining: 47.2s
675:	learn: 0.8255725	total: 1m 38s	remaining: 47.1s
676:	learn: 0.8256743	total: 1m 38s	remaining: 47s
677:	learn: 0.8258742	total: 1m 38s	remaining: 46.8s
678:	learn: 0.8259816	total: 1m 38s	remaining: 46.7s
679:	learn: 0.8261044	total: 1m 38s	remaining: 46.6s
680:	learn: 0.8261633	total: 1m 39s	remaining: 46.4s
681:	learn: 0.8261799	total: 1m 39s	remaining: 46.2s
682:	learn: 0.8262844	total: 1m 39s	remaining: 46.1s
683:	learn: 0.8264318	total: 1m 39s	remaining: 46s
684:	learn: 0.8265294	total: 1m 39s	remaining: 45.8s
685:	learn: 0.8266769	total: 1m 39s	remaining: 45.6s

823:	learn: 0.8417251	total: 1m 59s	remaining: 25.6s
824:	learn: 0.8418191	total: 1m 59s	remaining: 25.5s
825:	learn: 0.8419082	total: 2m	remaining: 25.3s
826:	learn: 0.8419971	total: 2m	remaining: 25.2s
827:	learn: 0.8421124	total: 2m	remaining: 25s
828:	learn: 0.8421505	total: 2m	remaining: 24.9s
829:	learn: 0.8422064	total: 2m	remaining: 24.7s
830:	learn: 0.8422603	total: 2m	remaining: 24.6s
831:	learn: 0.8422745	total: 2m	remaining: 24.4s
832:	learn: 0.8423551	total: 2m 1s	remaining: 24.3s
833:	learn: 0.8425082	total: 2m 1s	remaining: 24.1s
834:	learn: 0.8426130	total: 2m 1s	remaining: 24s
835:	learn: 0.8426652	total: 2m 1s	remaining: 23.8s
836:	learn: 0.8427979	total: 2m 1s	remaining: 23.7s
837:	learn: 0.8430066	total: 2m 1s	remaining: 23.6s
838:	learn: 0.8431158	total: 2m 2s	remaining: 23.4s
839:	learn: 0.8431499	total: 2m 2s	remaining: 23.3s
840:	learn: 0.8432754	total: 2m 2s	remaining: 23.1s
841:	learn: 0.8434116	total: 2m 2s	remaining: 23s
842:	learn: 0.8435749	total: 2m 2s	re

981:	learn: 0.8575476	total: 2m 21s	remaining: 2.6s
982:	learn: 0.8577026	total: 2m 22s	remaining: 2.46s
983:	learn: 0.8577651	total: 2m 22s	remaining: 2.31s
984:	learn: 0.8578212	total: 2m 22s	remaining: 2.17s
985:	learn: 0.8579037	total: 2m 22s	remaining: 2.02s
986:	learn: 0.8579004	total: 2m 22s	remaining: 1.88s
987:	learn: 0.8580469	total: 2m 22s	remaining: 1.73s
988:	learn: 0.8581549	total: 2m 22s	remaining: 1.59s
989:	learn: 0.8583487	total: 2m 22s	remaining: 1.44s
990:	learn: 0.8585142	total: 2m 22s	remaining: 1.3s
991:	learn: 0.8585849	total: 2m 23s	remaining: 1.15s
992:	learn: 0.8586454	total: 2m 23s	remaining: 1.01s
993:	learn: 0.8587793	total: 2m 23s	remaining: 865ms
994:	learn: 0.8590209	total: 2m 23s	remaining: 721ms
995:	learn: 0.8591210	total: 2m 23s	remaining: 576ms
996:	learn: 0.8592452	total: 2m 23s	remaining: 432ms
997:	learn: 0.8592922	total: 2m 23s	remaining: 288ms
998:	learn: 0.8593668	total: 2m 23s	remaining: 144ms
999:	learn: 0.8595000	total: 2m 24s	remaining: 0

#### give submission

In [165]:
xgb_model.fit(X_train, Y_train)
lgb_model.fit(X_train, Y_train)

xgb_predictions = xgb_model.predict_proba(X_test)[:, 1]
lgb_predictions = lgb_model.predict_proba(X_test)[:, 1]

X_test['xgb_preds'] = xgb_predictions
X_test['lgb_preds'] = lgb_predictions

In [166]:
xgb_predictions = []
lgb_predictions = []

for train, val in skf.split(X_train, Y_train):    
        
        xgb_model.fit(X_train.iloc[train], Y_train.iloc[train])
        lgb_model.fit(X_train.iloc[train], Y_train.iloc[train])

        xgb_prediction = xgb_model.predict_proba(X_train.iloc[val])[:, 1]
        lgb_prediction = lgb_model.predict_proba(X_train.iloc[val])[:, 1]

        xgb_predictions.append(xgb_prediction)
        lgb_predictions.append(lgb_prediction)
   
X_train['xgb_preds'] = 0
X_train['lgb_preds'] = 0

for i, fold in enumerate(skf.split(X_train, Y_train)):
    train, val = fold[0], fold[1]

    X_train.iloc[val, -2] = xgb_predictions[i]
    X_train.iloc[val, -1] = lgb_predictions[i]

In [167]:
ctb_model.fit(X_train, Y_train)
stacking_prediction = ctb_model.predict_proba(X_test)[:, 1]

0:	learn: 0.6950084	total: 181ms	remaining: 3m 1s
1:	learn: 0.7051834	total: 338ms	remaining: 2m 48s
2:	learn: 0.7119203	total: 543ms	remaining: 3m
3:	learn: 0.7148392	total: 715ms	remaining: 2m 58s
4:	learn: 0.7163388	total: 907ms	remaining: 3m
5:	learn: 0.7163210	total: 1.07s	remaining: 2m 57s
6:	learn: 0.7187585	total: 1.25s	remaining: 2m 58s
7:	learn: 0.7191771	total: 1.43s	remaining: 2m 57s
8:	learn: 0.7204376	total: 1.62s	remaining: 2m 58s
9:	learn: 0.7210430	total: 1.79s	remaining: 2m 57s
10:	learn: 0.7273781	total: 1.97s	remaining: 2m 57s
11:	learn: 0.7287631	total: 2.13s	remaining: 2m 55s
12:	learn: 0.7296076	total: 2.33s	remaining: 2m 56s
13:	learn: 0.7291649	total: 2.5s	remaining: 2m 56s
14:	learn: 0.7294049	total: 2.69s	remaining: 2m 56s
15:	learn: 0.7300499	total: 2.85s	remaining: 2m 55s
16:	learn: 0.7302703	total: 3.04s	remaining: 2m 55s
17:	learn: 0.7300274	total: 3.22s	remaining: 2m 55s
18:	learn: 0.7302164	total: 3.41s	remaining: 2m 56s
19:	learn: 0.7305525	total: 3.59

161:	learn: 0.7451009	total: 31.1s	remaining: 2m 40s
162:	learn: 0.7452742	total: 31.2s	remaining: 2m 40s
163:	learn: 0.7453102	total: 31.4s	remaining: 2m 40s
164:	learn: 0.7454116	total: 31.6s	remaining: 2m 39s
165:	learn: 0.7454511	total: 31.8s	remaining: 2m 39s
166:	learn: 0.7454709	total: 32s	remaining: 2m 39s
167:	learn: 0.7455724	total: 32.1s	remaining: 2m 39s
168:	learn: 0.7456312	total: 32.3s	remaining: 2m 39s
169:	learn: 0.7457207	total: 32.5s	remaining: 2m 38s
170:	learn: 0.7458073	total: 32.7s	remaining: 2m 38s
171:	learn: 0.7459312	total: 32.9s	remaining: 2m 38s
172:	learn: 0.7460340	total: 33.1s	remaining: 2m 38s
173:	learn: 0.7461029	total: 33.2s	remaining: 2m 37s
174:	learn: 0.7461820	total: 33.4s	remaining: 2m 37s
175:	learn: 0.7463327	total: 33.6s	remaining: 2m 37s
176:	learn: 0.7463506	total: 33.8s	remaining: 2m 37s
177:	learn: 0.7464467	total: 34s	remaining: 2m 36s
178:	learn: 0.7465246	total: 34.1s	remaining: 2m 36s
179:	learn: 0.7465886	total: 34.3s	remaining: 2m 3

318:	learn: 0.7610854	total: 59.8s	remaining: 2m 7s
319:	learn: 0.7611608	total: 59.9s	remaining: 2m 7s
320:	learn: 0.7612628	total: 1m	remaining: 2m 7s
321:	learn: 0.7613484	total: 1m	remaining: 2m 6s
322:	learn: 0.7614523	total: 1m	remaining: 2m 6s
323:	learn: 0.7615521	total: 1m	remaining: 2m 6s
324:	learn: 0.7616873	total: 1m	remaining: 2m 6s
325:	learn: 0.7617868	total: 1m 1s	remaining: 2m 6s
326:	learn: 0.7619124	total: 1m 1s	remaining: 2m 5s
327:	learn: 0.7620384	total: 1m 1s	remaining: 2m 5s
328:	learn: 0.7621364	total: 1m 1s	remaining: 2m 5s
329:	learn: 0.7622610	total: 1m 1s	remaining: 2m 5s
330:	learn: 0.7624361	total: 1m 1s	remaining: 2m 5s
331:	learn: 0.7625011	total: 1m 2s	remaining: 2m 4s
332:	learn: 0.7625907	total: 1m 2s	remaining: 2m 4s
333:	learn: 0.7627218	total: 1m 2s	remaining: 2m 4s
334:	learn: 0.7628249	total: 1m 2s	remaining: 2m 4s
335:	learn: 0.7629666	total: 1m 2s	remaining: 2m 4s
336:	learn: 0.7630662	total: 1m 2s	remaining: 2m 3s
337:	learn: 0.7631611	total

473:	learn: 0.7799686	total: 1m 28s	remaining: 1m 38s
474:	learn: 0.7800536	total: 1m 28s	remaining: 1m 38s
475:	learn: 0.7801812	total: 1m 28s	remaining: 1m 37s
476:	learn: 0.7803461	total: 1m 29s	remaining: 1m 37s
477:	learn: 0.7805140	total: 1m 29s	remaining: 1m 37s
478:	learn: 0.7805612	total: 1m 29s	remaining: 1m 37s
479:	learn: 0.7807160	total: 1m 29s	remaining: 1m 37s
480:	learn: 0.7808640	total: 1m 29s	remaining: 1m 36s
481:	learn: 0.7809728	total: 1m 29s	remaining: 1m 36s
482:	learn: 0.7810648	total: 1m 30s	remaining: 1m 36s
483:	learn: 0.7812141	total: 1m 30s	remaining: 1m 36s
484:	learn: 0.7813126	total: 1m 30s	remaining: 1m 36s
485:	learn: 0.7813845	total: 1m 30s	remaining: 1m 35s
486:	learn: 0.7815176	total: 1m 30s	remaining: 1m 35s
487:	learn: 0.7816533	total: 1m 31s	remaining: 1m 35s
488:	learn: 0.7817921	total: 1m 31s	remaining: 1m 35s
489:	learn: 0.7820660	total: 1m 31s	remaining: 1m 35s
490:	learn: 0.7821284	total: 1m 31s	remaining: 1m 34s
491:	learn: 0.7822464	total:

625:	learn: 0.7967364	total: 1m 55s	remaining: 1m 8s
626:	learn: 0.7968757	total: 1m 55s	remaining: 1m 8s
627:	learn: 0.7969447	total: 1m 55s	remaining: 1m 8s
628:	learn: 0.7969952	total: 1m 55s	remaining: 1m 8s
629:	learn: 0.7971401	total: 1m 55s	remaining: 1m 8s
630:	learn: 0.7972139	total: 1m 56s	remaining: 1m 7s
631:	learn: 0.7974024	total: 1m 56s	remaining: 1m 7s
632:	learn: 0.7974946	total: 1m 56s	remaining: 1m 7s
633:	learn: 0.7975942	total: 1m 56s	remaining: 1m 7s
634:	learn: 0.7976534	total: 1m 56s	remaining: 1m 7s
635:	learn: 0.7977530	total: 1m 56s	remaining: 1m 6s
636:	learn: 0.7978760	total: 1m 57s	remaining: 1m 6s
637:	learn: 0.7980006	total: 1m 57s	remaining: 1m 6s
638:	learn: 0.7981820	total: 1m 57s	remaining: 1m 6s
639:	learn: 0.7983224	total: 1m 57s	remaining: 1m 6s
640:	learn: 0.7984249	total: 1m 57s	remaining: 1m 6s
641:	learn: 0.7985420	total: 1m 58s	remaining: 1m 5s
642:	learn: 0.7986513	total: 1m 58s	remaining: 1m 5s
643:	learn: 0.7987778	total: 1m 58s	remaining:

782:	learn: 0.8119404	total: 2m 23s	remaining: 39.9s
783:	learn: 0.8119803	total: 2m 24s	remaining: 39.7s
784:	learn: 0.8120595	total: 2m 24s	remaining: 39.5s
785:	learn: 0.8121146	total: 2m 24s	remaining: 39.3s
786:	learn: 0.8121652	total: 2m 24s	remaining: 39.1s
787:	learn: 0.8123023	total: 2m 24s	remaining: 38.9s
788:	learn: 0.8124152	total: 2m 24s	remaining: 38.8s
789:	learn: 0.8124667	total: 2m 25s	remaining: 38.6s
790:	learn: 0.8125761	total: 2m 25s	remaining: 38.4s
791:	learn: 0.8127373	total: 2m 25s	remaining: 38.2s
792:	learn: 0.8127778	total: 2m 25s	remaining: 38s
793:	learn: 0.8128506	total: 2m 25s	remaining: 37.8s
794:	learn: 0.8129738	total: 2m 26s	remaining: 37.7s
795:	learn: 0.8130617	total: 2m 26s	remaining: 37.5s
796:	learn: 0.8131590	total: 2m 26s	remaining: 37.3s
797:	learn: 0.8132372	total: 2m 26s	remaining: 37.1s
798:	learn: 0.8133044	total: 2m 27s	remaining: 37s
799:	learn: 0.8134564	total: 2m 27s	remaining: 36.8s
800:	learn: 0.8135108	total: 2m 27s	remaining: 36.

939:	learn: 0.8262066	total: 2m 56s	remaining: 11.3s
940:	learn: 0.8262441	total: 2m 57s	remaining: 11.1s
941:	learn: 0.8263443	total: 2m 57s	remaining: 10.9s
942:	learn: 0.8264051	total: 2m 57s	remaining: 10.7s
943:	learn: 0.8264763	total: 2m 57s	remaining: 10.5s
944:	learn: 0.8266449	total: 2m 57s	remaining: 10.3s
945:	learn: 0.8267111	total: 2m 57s	remaining: 10.2s
946:	learn: 0.8268165	total: 2m 58s	remaining: 9.97s
947:	learn: 0.8270161	total: 2m 58s	remaining: 9.78s
948:	learn: 0.8270485	total: 2m 58s	remaining: 9.6s
949:	learn: 0.8271125	total: 2m 58s	remaining: 9.41s
950:	learn: 0.8271954	total: 2m 59s	remaining: 9.23s
951:	learn: 0.8272931	total: 2m 59s	remaining: 9.04s
952:	learn: 0.8274261	total: 2m 59s	remaining: 8.86s
953:	learn: 0.8275370	total: 2m 59s	remaining: 8.67s
954:	learn: 0.8276070	total: 3m	remaining: 8.48s
955:	learn: 0.8276779	total: 3m	remaining: 8.29s
956:	learn: 0.8277162	total: 3m	remaining: 8.11s
957:	learn: 0.8277743	total: 3m	remaining: 7.92s
958:	learn

In [168]:
min(stacking_prediction)

0.016784917882873996

In [169]:
write_pred('submission_Stacking.csv', stacking_prediction)